In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}
document.title='AiiDAlab QE app'

In [ ]:
from aiida import load_profile

load_profile();

In [ ]:
from aiidalab_widgets_base.utils.loaders import load_css

load_css(css_path="src/aiidalab_qe/app/static/styles")

In [ ]:
# Fix pybel import path
try:
    import sys

    sys.modules["pybel"] = __import__("openbabel", globals(), locals(), ["pybel"]).pybel
except Exception:
    pass

In [ ]:
from IPython.display import display

from aiidalab_qe.app.wrapper import AppWrapperContoller, AppWrapperModel, AppWrapperView

model = AppWrapperModel()
view = AppWrapperView()
controller = AppWrapperContoller(model, view)

display(view)

In [ ]:
import urllib.parse as urlparse

from aiidalab_qe.app.main import App
from aiidalab_widgets_base.bug_report import (
    install_create_github_issue_exception_handler,
)

install_create_github_issue_exception_handler(
    view.output,
    url="https://github.com/aiidalab/aiidalab-qe/issues/new",
    labels=("bug", "automated-report"),
)

url = urlparse.urlsplit(jupyter_notebook_url)  # noqa F821
query = urlparse.parse_qs(url.query)

app = App(qe_auto_setup=True)
# if a pk is provided in the query string, set it as the process of the app
if "pk" in query:
    pk = query["pk"][0]
    app.process = pk

view.main.children = [app]
view.app = app

In [ ]:
controller.enable_toggles()